In [ ]:
# https://www.quantitativo.com/p/a-mean-reversion-strategy-with-211

In [2]:
import vectorbt as vbt
import numpy as np
import talib as ta
import yfinance as yf

In [3]:
# Load data for Gold (XAUUSD)
symbol = 'QQQ'  # Gold Futures (1D timeframe)
#data = yf.download(symbol, start="2000-01-01", end="2024-01-01")
data = yf.download(symbol)
data[5:]

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-03-17,51.937500,52.000000,51.406250,51.562500,43.771824,3965000
1999-03-18,51.500000,52.593750,51.484375,52.562500,44.620750,4848400
1999-03-19,53.250000,53.250000,51.187500,51.218750,43.480026,7160400
1999-03-22,51.437500,51.562500,50.500000,50.593750,42.949451,5024800
1999-03-23,50.312500,50.500000,48.718750,48.812500,41.437344,10962400
...,...,...,...,...,...,...
2025-02-11,525.960022,529.919983,525.830017,527.989990,527.989990,19325500
2025-02-12,522.299988,529.190002,521.950012,528.299988,528.299988,25009400
2025-02-13,529.979980,536.219971,529.190002,535.900024,535.900024,28714800


In [5]:
# Define close price and compute the 200-period SMA
close = data['Close']
high = data['High']
low = data['Low']
hl = high - low
cl = close - low

rollingMean = hl.rolling(window=25).mean()
ibs = cl / hl
rollingHigh10 = high.rolling(window=10).max()
lowerBand = rollingHigh10 - 2.5 * rollingMean
sma_200 = close.rolling(window=200).mean()

In [15]:
# Generate entry and exit signals
if close < lowerBand and ibs < 0.3:
    entries = True
else:
    entries = False
#entries = (close < lowerBand and ibs < 0.3) # Go long whenever SPY closes under the lower band (3rd bullet) and IBS is lower than 0.3
exits = close[-1] > high[-2]  # Sell when close crosses below the 200 SMA

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [11]:
# Create the portfolio using the signals
portfolio = vbt.Portfolio.from_signals(close, entries, exits)

In [12]:
# Analyze the portfolio
portfolio_stats = portfolio.stats()
print(portfolio_stats)

Start                         1999-03-10 00:00:00
End                           2025-02-18 00:00:00
Period                                       6527
Start Value                                 100.0
End Value                             1035.218261
Total Return [%]                       935.218261
Benchmark Return [%]                   953.591168
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        82.963907
Max Drawdown Duration                      4136.0
Total Trades                                    1
Total Closed Trades                             0
Total Open Trades                               1
Open Trade PnL                         935.218261
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


c:\Coding\Algotrading\VectorBT\vectorbt-strategies\.venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Coding\Algotrading\VectorBT\vectorbt-strategies\.venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Coding\Algotrading\VectorBT\vectorbt-strategies\.venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Coding\Algotrading\VectorBT\vectorbt-strategies\.venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


In [7]:
# Plot the portfolio
portfolio.plot().show()